Use the checkpoint from before and train on the IMDB dataset

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the best model from Task 5
best_model = MLP(input_size=768)
best_model.load_state_dict(torch.load("best_checkpoint.pt"))
best_model.to(device)

# Define loss function & optimizer with lower learning rate
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(best_model.parameters(), lr=0.0001)


We have choosen 200 samples from imdb also for continuous learning

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Load IMDB dataset
imdb_df = pd.read_csv("https://raw.githubusercontent.com/Ankit152/IMDB-sentiment-analysis/master/IMDB-Dataset.csv")

# Convert sentiment labels to binary (0 = negative, 1 = positive)
imdb_df["label"] = imdb_df["sentiment"].map({"positive": 1, "negative": 0})
imdb_df.drop(columns=["sentiment"], inplace=True)

# Select 200 random samples for training, validation, and test
num_samples = 200
train_texts, test_texts, train_labels, test_labels = train_test_split(
    imdb_df["review"], imdb_df["label"], test_size=0.2, random_state=42, stratify=imdb_df["label"]
)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.25, random_state=42, stratify=train_labels
)

# Take only the first 200 samples
train_subset_texts, train_subset_labels = train_texts[:num_samples], train_labels[:num_samples]
val_subset_texts, val_subset_labels = val_texts[:num_samples], val_labels[:num_samples]
test_subset_texts, test_subset_labels = test_texts[:num_samples], test_labels[:num_samples]

# Convert text to embeddings
embedding_model = TextEmbeddings()

X_train_imdb = torch.tensor(embedding_model.get_embedding(train_subset_texts.tolist()), dtype=torch.float32)
X_val_imdb = torch.tensor(embedding_model.get_embedding(val_subset_texts.tolist()), dtype=torch.float32)
X_test_imdb = torch.tensor(embedding_model.get_embedding(test_subset_texts.tolist()), dtype=torch.float32)

y_train_imdb = torch.tensor(train_subset_labels.values, dtype=torch.long)
y_val_imdb = torch.tensor(val_subset_labels.values, dtype=torch.long)
y_test_imdb = torch.tensor(test_subset_labels.values, dtype=torch.long)

# Create PyTorch DataLoaders
batch_size = 16
train_loader_imdb = DataLoader(TensorDataset(X_train_imdb, y_train_imdb), batch_size=batch_size, shuffle=True)
val_loader_imdb = DataLoader(TensorDataset(X_val_imdb, y_val_imdb), batch_size=batch_size, shuffle=False)
test_loader_imdb = DataLoader(TensorDataset(X_test_imdb, y_test_imdb), batch_size=batch_size, shuffle=False)


In [ ]:
from torch.utils.tensorboard import SummaryWriter

# Create a separate TensorBoard writer for Task 6 logs
writer_imdb = SummaryWriter("runs/IMDB_Training")

def continual_learning(model, train_loader, val_loader, checkpoint_path, dataset_name, num_epochs=5):
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")

        # Training phase
        model.train()
        correct, total, train_loss = 0, 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            predicted = torch.argmax(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total

        # Validation phase
        model.eval()
        correct, total, val_loss = 0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                predicted = torch.argmax(outputs, dim=1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        val_acc = correct / total

        # Log Training & Validation Metrics to Separate TensorBoard
        writer_imdb.add_scalar(f"Loss/Train_{dataset_name}", train_loss / len(train_loader), epoch)
        writer_imdb.add_scalar(f"Loss/Validation_{dataset_name}", val_loss / len(val_loader), epoch)
        writer_imdb.add_scalar(f"Accuracy/Train_{dataset_name}", train_acc, epoch)
        writer_imdb.add_scalar(f"Accuracy/Validation_{dataset_name}", val_acc, epoch)

        # Save the best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), checkpoint_path)

    writer_imdb.flush()
    print(f"\nBest Validation Accuracy on {dataset_name}: {best_val_acc:.4f}")
    print(f"Continual learning model saved at '{checkpoint_path}'")

# Train on IMDB dataset
checkpoint_imdb = "checkpoint_imdb.pt"
continual_learning(best_model, train_loader_imdb, val_loader_imdb, checkpoint_imdb, "IMDB", num_epochs=5)

# Close the separate writer
writer_imdb.close()



Epoch 1/5

Epoch 2/5

Epoch 3/5

Epoch 4/5

Epoch 5/5

Best Validation Accuracy on IMDB: 0.7400
Continual learning model saved at 'checkpoint_imdb.pt'


In [ ]:
def evaluate_model(model, test_loader, dataset_name):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = torch.argmax(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    test_acc = correct / total
    print(f"\nFinal Test Accuracy on {dataset_name}: {test_acc:.4f}")

    # Log final test accuracy in Separate TensorBoard
    writer_imdb.add_scalar(f"Accuracy/Test_{dataset_name}", test_acc)
    writer_imdb.flush()

# Evaluate on IMDB
evaluate_model(best_model, test_loader_imdb, "IMDB")

# Close the writer after evaluation
writer_imdb.close()



Final Test Accuracy on IMDB: 0.7800


Compute the validation loss and accuracy on the validation set of the Dataset 1 and IMDB dataset

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load separate models for each dataset
best_model_sst2 = MLP(input_size=10000)  # Model for SST2 (BoW)
best_model_sst2.load_state_dict(torch.load("checkpoint_bow.pt"))
best_model_sst2.to(device)

best_model_imdb = MLP(input_size=768)  # Model for IMDB (Embeddings)
best_model_imdb.load_state_dict(torch.load("checkpoint_imdb.pt"))
best_model_imdb.to(device)

# Create a separate TensorBoard writer for Task 7 logs
writer_eval = SummaryWriter("runs/Evaluation")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def log_confusion_matrix(model, val_loader, dataset_name):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute confusion matrix
    cm = confusion_matrix(all_labels, all_preds)

    # Plot confusion matrix
    fig, ax = plt.subplots(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", ax=ax)
    ax.set_xlabel("Predicted Labels")
    ax.set_ylabel("True Labels")
    ax.set_title(f"Confusion Matrix - {dataset_name}")

    # Log to TensorBoard (Separate from Training Logs)
    writer_eval.add_figure(f"Confusion Matrix/{dataset_name}", fig)
    writer_eval.flush()
    plt.close(fig)

    print(f"Confusion matrix logged for {dataset_name}")


In [ ]:
# Log Confusion Matrix for SST2 dataset (BoW Model)
log_confusion_matrix(best_model_sst2, val_loader, "SST2")

# Log Confusion Matrix for IMDB dataset (Embedding Model)
log_confusion_matrix(best_model_imdb, val_loader_imdb, "IMDB")

writer_eval.close()


Confusion matrix logged for SST2
Confusion matrix logged for IMDB


In [ ]:
import torch
import os

def save_compressed_checkpoint(model, filepath):
    """Save the model in a compressed format."""
    state_dict = model.state_dict()
    torch.save(state_dict, filepath, _use_new_zipfile_serialization=True)
    print(f" Compressed checkpoint saved at: {filepath}")

# Apply compression in Task 5 (SST2 Checkpoint)
save_compressed_checkpoint(best_model_sst2, "compressed_checkpoint_sst2.pt")

# Apply compression in Task 6 (IMDB Checkpoint)
save_compressed_checkpoint(best_model_imdb, "compressed_checkpoint_imdb.pt")

# Verify file sizes
print(f"Compressed SST2 Checkpoint Size: {os.path.getsize('compressed_checkpoint_sst2.pt') / 1024:.2f} KB")
print(f"Compressed IMDB Checkpoint Size: {os.path.getsize('compressed_checkpoint_imdb.pt') / 1024:.2f} KB")


 Compressed checkpoint saved at: compressed_checkpoint_sst2.pt
 Compressed checkpoint saved at: compressed_checkpoint_imdb.pt
Compressed SST2 Checkpoint Size: 20680.50 KB
Compressed IMDB Checkpoint Size: 2216.50 KB


Tensorboard

In [ ]:
%load_ext tensorboard


In [ ]:
%tensorboard --logdir=runs/Text_Classification


In [ ]:
%tensorboard --logdir=runs/Training


In [ ]:
%tensorboard --logdir=runs/IMDB_Training


In [ ]:
%tensorboard --logdir=runs/Evaluation


In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter

# Load trained models from previous tasks
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

best_model_sst2 = MLP(input_size=10000)  # Model for SST2 (BoW)
best_model_sst2.load_state_dict(torch.load("compressed_checkpoint_sst2.pt"))
best_model_sst2.to(device)

best_model_imdb = MLP(input_size=768)  # Model for IMDB (Embeddings)
best_model_imdb.load_state_dict(torch.load("compressed_checkpoint_imdb.pt"))
best_model_imdb.to(device)

# Create a separate TensorBoard writer for final metrics
writer = SummaryWriter("runs/Final_Metrics")


In [ ]:
def compute_final_loss_accuracy(model, data_loader):
    model.eval()
    correct, total, total_loss = 0, 0, 0
    criterion = torch.nn.CrossEntropyLoss()  # Same loss function used in training

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            # Compute loss
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Compute accuracy
            predicted = torch.argmax(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(data_loader)
    accuracy = correct / total

    return avg_loss, accuracy


In [ ]:
# Compute final loss & accuracy for SST2 dataset
final_train_loss_sst2, final_train_acc_sst2 = compute_final_loss_accuracy(best_model_sst2, train_loader)
final_val_loss_sst2, final_val_acc_sst2 = compute_final_loss_accuracy(best_model_sst2, val_loader)
final_test_loss_sst2, final_test_acc_sst2 = compute_final_loss_accuracy(best_model_sst2, test_loader)

# Compute final loss & accuracy for IMDB dataset
final_train_loss_imdb, final_train_acc_imdb = compute_final_loss_accuracy(best_model_imdb, train_loader_imdb)
final_val_loss_imdb, final_val_acc_imdb = compute_final_loss_accuracy(best_model_imdb, val_loader_imdb)
final_test_loss_imdb, final_test_acc_imdb = compute_final_loss_accuracy(best_model_imdb, test_loader_imdb)

# Log Loss Metrics in TensorBoard
writer.add_scalar("Final Loss/SST2 Train", final_train_loss_sst2)
writer.add_scalar("Final Loss/SST2 Validation", final_val_loss_sst2)
writer.add_scalar("Final Loss/SST2 Test", final_test_loss_sst2)

writer.add_scalar("Final Loss/IMDB Train", final_train_loss_imdb)
writer.add_scalar("Final Loss/IMDB Validation", final_val_loss_imdb)
writer.add_scalar("Final Loss/IMDB Test", final_test_loss_imdb)

# Log Accuracy Metrics in TensorBoard
writer.add_scalar("Final Accuracy/SST2 Train", final_train_acc_sst2)
writer.add_scalar("Final Accuracy/SST2 Validation", final_val_acc_sst2)
writer.add_scalar("Final Accuracy/SST2 Test", final_test_acc_sst2)

writer.add_scalar("Final Accuracy/IMDB Train", final_train_acc_imdb)
writer.add_scalar("Final Accuracy/IMDB Validation", final_val_acc_imdb)
writer.add_scalar("Final Accuracy/IMDB Test", final_test_acc_imdb)

# Log Everything as Text in TensorBoard for Easy Screenshot
final_metrics_text = f"""
SST2 Dataset:
- Final Train Loss: {final_train_loss_sst2:.4f} | Final Train Accuracy: {final_train_acc_sst2:.4f}
- Final Validation Loss: {final_val_loss_sst2:.4f} | Final Validation Accuracy: {final_val_acc_sst2:.4f}
- Final Test Loss: {final_test_loss_sst2:.4f} | Final Test Accuracy: {final_test_acc_sst2:.4f}

IMDB Dataset:
- Final Train Loss: {final_train_loss_imdb:.4f} | Final Train Accuracy: {final_train_acc_imdb:.4f}
- Final Validation Loss: {final_val_loss_imdb:.4f} | Final Validation Accuracy: {final_val_acc_imdb:.4f}
- Final Test Loss: {final_test_loss_imdb:.4f} | Final Test Accuracy: {final_test_acc_imdb:.4f}
"""

writer.add_text("Final Metrics", final_metrics_text)

writer.flush()
writer.close()

print("Final loss and accuracy logged in TensorBoard without retraining.")


Final loss and accuracy logged in TensorBoard without retraining.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs/Final_Metrics
